<a href="https://colab.research.google.com/github/Classymotion/pytorch-CycleGAN-and-pix2pix/blob/master/qna.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
test